In [1]:
import pandas as pd
esnli_test = pd.read_csv('esnli_test.csv')[:500]
esnli_test.shape

(500, 19)

In [32]:
import pandas as pd
df = pd.read_csv('explanations_post_redundant_v3.csv')
df.shape
df = df.rename(columns={'explanation': 'corrected_explantion'}) 

In [33]:
df.head()

,gold_label,premise,hypothesis,machine_explanation,corrected_explantion
0,1,this church choir sings to the masses as they ...,the church has cracks in the ceiling,the roof is falling down.The church was built...,the roof is falling down.The church was built...
1,0,this church choir sings to the masses as they ...,the church is filled with song,the choir is singing joyously.The church has ...,the choir is singing joyously.The church has ...
2,2,this church choir sings to the masses as they ...,a choir singing at a baseball game,the church is singing to a different choir.Th...,the church is singing to a different choir.Th...
3,1,a woman with a green headscarf blue shirt and ...,the woman is young,she is a girl.The woman in the picture is the...,she is a girl.The woman in the picture is the...
4,0,a woman with a green headscarf blue shirt and ...,the woman is very happy,she is a Muslim.The woman's face is covered b...,she is a Muslim.The woman's face is covered b...


In [34]:
# Assuming both DataFrames are indexed the same way or there is a common 'id' column
df = df.join(esnli_test[['Explanation_1', 'Explanation_2', 'Explanation_3']])

In [11]:
pip install rouge-score bert-score POT gensim

Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import pandas as pd
from gensim.models import KeyedVectors
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score

from gensim.models import KeyedVectors
from gensim.downloader import load

def load_word2vec_model():
    print("Checking for the Google News model locally...")
    try:
        # This will load the model from the gensim-data repository if not locally available
        model = load('word2vec-google-news-300')
        print("Model is loaded.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return model

# Usage
model = load_word2vec_model()

import nltk
nltk.download('wordnet')

Checking for the Google News model locally...
Model is loaded.


[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
from nltk.translate.bleu_score import SmoothingFunction

def evaluate_explanations(df_eval, col_machine, model):
    from bert_score import score
    from nltk.translate.bleu_score import SmoothingFunction
    import numpy as np

    smoothie = SmoothingFunction().method1  # Simple additive smoothing.
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    
    # Prepare lists to store scores
    bleu_scores = []
    meteor_scores = []
    rouge1_scores = []
    rougeL_scores = []
    bert_precision = []
    bert_recall = []
    bert_f1 = []
    wmd_scores = []

    # Iterate through DataFrame rows
    for index, row in df_eval.iterrows():
        if pd.isna(row[col_machine]) or any(pd.isna(row[exp_col]) for exp_col in ['Explanation_1', 'Explanation_2', 'Explanation_3']):
            continue  # Skip this row if any essential column has NaN
        
        candidate = row[col_machine].split()
        references = []
        candidates = []
        
        scores_per_exp = {
            'BLEU': [],
            'METEOR': [],
            'ROUGE1': [],
            'ROUGEL': [],
            'BERT_P': [],
            'BERT_R': [],
            'BERT_F1': [],
            'WMD': []
        }

        for exp_col in ['Explanation_1', 'Explanation_2', 'Explanation_3']:
            reference = row[exp_col].split()
            references.append(' '.join(reference))
            candidates.append(' '.join(candidate))
            
            scores_per_exp['BLEU'].append(sentence_bleu([reference], candidate, smoothing_function=smoothie, weights=(0.5, 0.5, 0, 0)))
            scores_per_exp['METEOR'].append(meteor_score([reference], candidate))
            rouge_scores = rouge_scorer_instance.score(' '.join(reference), ' '.join(candidate))
            scores_per_exp['ROUGE1'].append(rouge_scores['rouge1'].fmeasure)
            scores_per_exp['ROUGEL'].append(rouge_scores['rougeL'].fmeasure)
            scores_per_exp['WMD'].append(model.wmdistance(reference, candidate))

        P, R, F1 = score(candidates, references, lang="en", verbose=True)
        scores_per_exp['BERT_P'].extend(P.tolist())
        scores_per_exp['BERT_R'].extend(R.tolist())
        scores_per_exp['BERT_F1'].extend(F1.tolist())

        bleu_scores.append(np.mean(scores_per_exp['BLEU']))
        meteor_scores.append(np.mean(scores_per_exp['METEOR']))
        rouge1_scores.append(np.mean(scores_per_exp['ROUGE1']))
        rougeL_scores.append(np.mean(scores_per_exp['ROUGEL']))
        bert_precision.append(np.mean(scores_per_exp['BERT_P']))
        bert_recall.append(np.mean(scores_per_exp['BERT_R']))
        bert_f1.append(np.mean(scores_per_exp['BERT_F1']))
        wmd_scores.append(np.mean(scores_per_exp['WMD']))

    df_eval = df_eval.dropna(subset=[col_machine, 'Explanation_1', 'Explanation_2', 'Explanation_3'])
    df_eval['BLEU'] = bleu_scores
    df_eval['METEOR'] = meteor_scores
    df_eval['ROUGE1'] = rouge1_scores
    df_eval['ROUGEL'] = rougeL_scores
    df_eval['BERTScore_Precision'] = bert_precision
    df_eval['BERTScore_Recall'] = bert_recall
    df_eval['BERTScore_F1'] = bert_f1
    df_eval['WMD'] = wmd_scores

    overall_scores = {
        'Overall_BLEU': np.mean(bleu_scores),
        'Overall_METEOR': np.mean(meteor_scores),
        'Overall_ROUGE1': np.mean(rouge1_scores),
        'Overall_ROUGEL': np.mean(rougeL_scores),
        'Overall_BERT_Precision': np.mean(bert_precision),
        'Overall_BERT_Recall': np.mean(bert_recall),
        'Overall_BERT_F1': np.mean(bert_f1),
        'Overall_WMD': np.mean(wmd_scores)
    }

    return df_eval, overall_scores

# Usage
df_eval = df.copy()
df_eval, overall_scores = evaluate_explanations(df_eval, 'machine_explanation', model)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.83 seconds, 3.61 sentences/sec


In [36]:
print("hello_world!")

hello_world!


In [37]:
df_eval.head()

,gold_label,premise,hypothesis,machine_explanation,corrected_explantion,Explanation_1,Explanation_2,Explanation_3,BLEU,METEOR,ROUGE1,ROUGEL,BERTScore_Precision,BERTScore_Recall,BERTScore_F1,WMD
0,1,this church choir sings to the masses as they ...,the church has cracks in the ceiling,the roof is falling down.The church was built...,the roof is falling down.The church was built...,Not all churches have cracks in the ceiling,There is no indication that there are cracks i...,Not all churches have cracks in the ceiling.,0.015195,0.136006,0.157576,0.144242,0.848680,0.897106,0.872210,0.950382
1,0,this church choir sings to the masses as they ...,the church is filled with song,the choir is singing joyously.The church has ...,the choir is singing joyously.The church has ...,"""Filled with song"" is a rephrasing of the ""cho...",hearing song brings joyous in the church.,If the church choir sings then the church is f...,0.042363,0.209105,0.266466,0.225083,0.873502,0.887632,0.880244,0.877671
2,2,this church choir sings to the masses as they ...,a choir singing at a baseball game,the church is singing to a different choir.Th...,the church is singing to a different choir.Th...,A choir sing some other songs other than book ...,The choir is at a chruch not a baseball game.,A baseball game isn’t played at a church.,0.016191,0.155776,0.127005,0.120202,0.847699,0.860797,0.854120,1.036275
3,1,a woman with a green headscarf blue shirt and ...,the woman is young,she is a girl.The woman in the picture is the...,she is a girl.The woman in the picture is the...,the woman could've been old rather than young,There is no indication that the woman is young.,Not all women are young.,0.014263,0.120863,0.094862,0.072793,0.818126,0.862428,0.839659,1.003511
4,0,a woman with a green headscarf blue shirt and ...,the woman is very happy,she is a Muslim.The woman's face is covered b...,she is a Muslim.The woman's face is covered b...,a grin suggests hapiness.,A WOMAN WITH BIG GRIN IS HAPPY,One must be happy in order to have a big grin.,0.003491,0.158730,0.057309,0.050365,0.804395,0.857471,0.830066,1.222058


In [38]:
import pandas as pd

# Assuming 'df' is your DataFrame
df_eval.to_csv('explanations_eval_v3_redundant.csv', index=False)  # Convert DataFrame to CSV

In [39]:
overall_scores

{'Overall_BLEU': 0.024495715225961525,
 'Overall_METEOR': 0.16221057358756635,
 'Overall_ROUGE1': 0.1506256013024817,
 'Overall_ROUGEL': 0.1138924105790355,
 'Overall_BERT_Precision': 0.8263227093617121,
 'Overall_BERT_Recall': 0.859371411760648,
 'Overall_BERT_F1': 0.8423890387614569,
 'Overall_WMD': 1.0058155752631905}